In [ ]:
import cv2
from deepface import DeepFace
import os
import time
import numpy as np
from collections import Counter
import tensorflow as tf
import csv # Added for CSV logging

print("Libraries imported successfully.")

# --- GPU Check ---
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"GPU(s) found and configured: {len(gpus)}")
        print("DeepFace will now use the GPU for processing.")
    except RuntimeError as e:
        print(e)
else:
    print("No GPU found. DeepFace will use the CPU. Processing will be slower.")
    print("For faster performance, please ensure you have a compatible NVIDIA GPU, CUDA, cuDNN, and the correct TensorFlow package installed.")

# ==============================================================================
# Step 3: Define the core analysis and visualization functions
# ==============================================================================
def analyze_frame_attributes(frame):
    """
    This function takes a single video frame (as a numpy array), detects faces,
    and analyzes age, gender, and emotion for each face.
    """
    try:
        results = DeepFace.analyze(
            img_path=frame,
            actions=['age', 'gender', 'emotion'],
            enforce_detection=False,
            detector_backend='retinaface'
        )
        return results
    except Exception as e:
        return []

def draw_results_on_frame(frame, results):
    """
    Draws the analysis results (bounding boxes, text) on the given frame.
    """
    annotated_frame = frame.copy()

    for face_data in results:
        box = face_data['region']
        x, y, w, h = box['x'], box['y'], box['w'], box['h']

        gender = face_data['dominant_gender']
        age = face_data['age']
        emotion = face_data['dominant_emotion']

        color = (255, 0, 0) if gender == 'Man' else (203, 192, 255)

        cv2.rectangle(annotated_frame, (x, y), (x + w, y + h), color, 2)

        text_label = f"{age}, {gender}, {emotion}"

        (text_width, text_height), _ = cv2.getTextSize(text_label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
        cv2.rectangle(annotated_frame, (x, y - text_height - 10), (x + text_width + 4, y), color, -1)
        cv2.putText(annotated_frame, text_label, (x + 2, y - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)

    return annotated_frame

def create_stats_display(num_people, avg_age, gender_counts, emotion_counts, excitement_index_men, excitement_index_women, window_width=400, window_height=350):
    """Creates a display image for crowd statistics, now with gender-specific excitement indexes."""
    display = np.zeros((window_height, window_width, 3), dtype=np.uint8)
    display[:] = (40, 40, 40) # Dark gray background

    cv2.putText(display, "Crowd Statistics", (window_width // 2 - 100, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

    cv2.putText(display, f"People Detected: {num_people}", (20, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    avg_age_text = f"Average Age: {avg_age:.1f}" if num_people > 0 else "Average Age: N/A"
    cv2.putText(display, avg_age_text, (20, 105),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

    men_count = gender_counts.get('Man', 0)
    women_count = gender_counts.get('Woman', 0)
    cv2.putText(display, f"Men: {men_count}", (20, 140),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    cv2.putText(display, f"Women: {women_count}", (180, 140),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (203, 192, 255), 2)

    cv2.putText(display, "Emotion Breakdown:", (20, 175),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    top_emotions = emotion_counts.most_common(2)
    y_offset = 205
    for emotion, count in top_emotions:
        emotion_text = f"- {emotion}: {count}"
        cv2.putText(display, emotion_text, (25, y_offset),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (200, 200, 200), 2)
        y_offset += 25

    # --- MODIFIED: Excitement Index Display for both genders ---
    y_offset += 15
    cv2.putText(display, f"Men Excitement: {excitement_index_men:.2f}", (20, y_offset),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 150, 150), 2) # Light Blue
    y_offset += 30
    cv2.putText(display, f"Women Excitement: {excitement_index_women:.2f}", (20, y_offset),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (203, 192, 255), 2) # Pink

    return display

# ==============================================================================
# Step 4: Define video paths and run the analysis pipeline
# ==============================================================================
video_path = r"C:\vid_path"
output_video_path = r"output_video_with_stats_retinaface.mp4"
# --- ADDED: Define output CSV file path ---
output_csv_path = r"crowd_analysis_log.csv"

if not os.path.exists(video_path):
    print(f"ERROR: The video file was not found at the specified path: {video_path}")
else:
    print(f"Found video, starting analysis on: {video_path}")
    print(f"Statistical logs will be saved to: {output_csv_path}")

    cap = cv2.VideoCapture(video_path)

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    if not cap.isOpened():
        print("Error: Could not open video stream or file")
        exit()

    # --- ADDED: Setup CSV file for writing ---
    csv_headers = [
        'Timestamp (s)', 'Frame', 'Total People', 'Avg Age',
        'Men Count', 'Women Count', 'Excitement Index (Men)', 'Excitement Index (Women)',
        'Dominant Emotion', 'Dominant Emotion Count'
    ]
    with open(output_csv_path, 'w', newline='') as csv_file:
        csv_writer = csv.writer(csv_file)
        csv_writer.writerow(csv_headers) # Write the header row

        cv2.namedWindow('Video Analysis', cv2.WINDOW_NORMAL)
        cv2.namedWindow('Crowd Statistics', cv2.WINDOW_NORMAL)

        frame_count = 0
        process_interval = 15
        last_results = []
        start_time = time.time()

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            frame_count += 1

            if frame_count % process_interval == 0:
                print(f"Processing frame {frame_count} / {total_frames}...")
                current_results = analyze_frame_attributes(frame)
                if current_results:
                    last_results = current_results

                # --- ADDED: Log data to CSV only when analysis is run ---
                num_people = len(last_results)
                if num_people > 0:
                    total_age = sum(face['age'] for face in last_results)
                    avg_age = total_age / num_people

                    gender_list = [face.get('dominant_gender', 'Unknown') for face in last_results]
                    emotion_list = [face.get('dominant_emotion', 'Unknown') for face in last_results]
                    gender_counts = Counter(gender_list)
                    emotion_counts = Counter(emotion_list)

                    men_count = gender_counts.get('Man', 0)
                    women_count = gender_counts.get('Woman', 0)

                    # Calculate gender-specific excitement
                    positive_emotions = ['happy', 'surprise']
                    positive_men = sum(1 for face in last_results if face.get('dominant_gender') == 'Man' and face.get('dominant_emotion') in positive_emotions)
                    positive_women = sum(1 for face in last_results if face.get('dominant_gender') == 'Woman' and face.get('dominant_emotion') in positive_emotions)

                    excitement_index_men = positive_men / men_count if men_count > 0 else 0.0
                    excitement_index_women = positive_women / women_count if women_count > 0 else 0.0

                    # Prepare data for CSV row
                    timestamp_sec = frame_count / fps
                    top_emotion_name = emotion_counts.most_common(1)[0][0] if emotion_counts else "N/A"
                    top_emotion_count = emotion_counts.most_common(1)[0][1] if emotion_counts else 0

                    data_row = [
                        f"{timestamp_sec:.2f}", frame_count, num_people, f"{avg_age:.1f}",
                        men_count, women_count, f"{excitement_index_men:.2f}", f"{excitement_index_women:.2f}",
                        top_emotion_name, top_emotion_count
                    ]
                    csv_writer.writerow(data_row)

            # --- Calculate Statistics (for display on every frame) ---
            num_people = len(last_results)
            total_age = 0
            gender_list = []
            emotion_list = []

            # --- MODIFIED: Calculate two excitement indexes ---
            positive_emotions = ['happy', 'surprise']
            positive_men_count = 0
            positive_women_count = 0

            for face_data in last_results:
                total_age += face_data['age']
                gender = face_data.get('dominant_gender', 'Unknown')
                emotion = face_data.get('dominant_emotion', 'Unknown')
                gender_list.append(gender)
                emotion_list.append(emotion)

                if gender == 'Man' and emotion in positive_emotions:
                    positive_men_count += 1
                elif gender == 'Woman' and emotion in positive_emotions:
                    positive_women_count += 1

            avg_age = total_age / num_people if num_people > 0 else 0
            gender_counts = Counter(gender_list)
            emotion_counts = Counter(emotion_list)

            men_count = gender_counts.get('Man', 0)
            women_count = gender_counts.get('Woman', 0)

            excitement_index_men = positive_men_count / men_count if men_count > 0 else 0.0
            excitement_index_women = positive_women_count / women_count if women_count > 0 else 0.0

            # --- Create and Display Outputs ---
            # Pass the two new indexes to the display function
            stats_frame = create_stats_display(num_people, avg_age, gender_counts, emotion_counts, excitement_index_men, excitement_index_women)
            cv2.imshow('Crowd Statistics', stats_frame)

            annotated_frame = draw_results_on_frame(frame, last_results)
            cv2.imshow('Video Analysis', annotated_frame)

            video_writer.write(annotated_frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        end_time = time.time()
        print("\nVideo processing complete.")
        print(f"Total time taken: {end_time - start_time:.2f} seconds.")
        print(f"Annotated video saved to: {output_video_path}")
        print(f"Analysis log saved to: {output_csv_path}")

        cap.release()
        video_writer.release()
        cv2.destroyAllWindows()